In [1]:
import pandas as pd
import json
from ast import literal_eval
from pandas.io.json import json_normalize
from datetime import datetime
import re
import requests
import numpy as np

KIRSTEN - FRANCHISE TABLE

In [2]:
#Pull website data and extract table
url = 'https://www.the-numbers.com/movies/franchises'

response = requests.get(url)

tables = pd.read_html(response.text)

f_all_data_df = tables[0]

In [3]:
#Keep relevant rows: Franchise name and Worldwide Box Office
franchise_df = f_all_data_df[["Franchise", "Worldwide Box Office"]]

In [4]:
#Remove currency formatting and drop franchises with no value or 0  
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace(',', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace('$', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].astype(float)

franchise_df.replace(0, np.nan, inplace=True)
franchise_df.dropna(inplace=True)

C:\Users\Lealo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Lealo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lealo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [5]:
#Export franchise data
#franchise_df.to_csv('franchise_df.csv', index=False)

In [6]:
#Read metadata CSV
csv_file = "movies_metadata.csv"

metadata_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape', low_memory=False)

In [7]:
#Keep relevant rows: original_title, collection, revenue. Drop rows with 0 value
franchise_single = metadata_df[['original_title', 'belongs_to_collection', 'revenue']]

franchise_single.astype({'revenue': 'float64'}).dtypes

franchise_single = franchise_single.query("revenue > 0")
franchise_single = franchise_single.reset_index(drop=True)
franchise_single.dropna(inplace=True)

In [8]:
#Export data for individual movies
#franchise_single.to_csv('metadata_franchise.csv', index=False)

In [9]:
#Create empty dataframe for final table
final_table = pd.DataFrame (columns = ['Franchise', 'Movie Title','Movie Revenue', 
                                       'Total Franchise Revenue', '% of Total Franchise Revenue'])

In [10]:
#Match if collection contains franchise name and input desired data into final table
for idx, fn in franchise_df.iterrows():
    search = fn["Franchise"]
    wwbo = fn["Worldwide Box Office"]
    title_match = franchise_single["belongs_to_collection"].str.contains(search)
    indexes = (title_match[title_match==True].index.values)
    
    revenue = franchise_single.loc[indexes, 'revenue'] 
    movie_title = franchise_single.loc[indexes, 'original_title'] 
    franchise_performance = (revenue / wwbo)*100

    final_data = {'Franchise': search, 'Movie Title': movie_title, 'Movie Revenue': revenue, 
                  'Total Franchise Revenue': wwbo, '% of Total Franchise Revenue': franchise_performance}
    final_table = final_table.append(pd.DataFrame(final_data, columns= ['Franchise', 'Movie Title', 
                                                                        'Movie Revenue', 'Total Franchise Revenue', 
                                                                        '% of Total Franchise Revenue']),ignore_index=True)

C:\Users\Lealo\Anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [11]:
#Clean final table
delete='[REC]'

clean_final = final_table[final_table.Franchise != delete]

#Export final table to CSV
clean_final.to_csv('Franchise_Performance_By_Movie.csv', index=False)

LEA- MOVIES TABLE, RATINGS TABLE

In [12]:
csv_file = "movies_metadata.csv"

ratings_data = "movie_ratings.csv"

# df = pd.read_csv(csv_file,'unicode_escape')

movies_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape')

ratings_df = pd.read_csv(ratings_data)

# csv_file.encode('utf-8').strip(csv_file)

C:\Users\Lealo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# df.head()

In [14]:
movies_df["genres"].head(1)

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
Name: genres, dtype: object

In [15]:
 # Step 2: Isolation of columns that are relevant
movie_genres = movies_df[['id', 'genres']]

In [16]:
# Step 3: Iterate over each row from our isolated dataset
# to_string converts the movie_genre dataframe to a string. movie_genre is a variable prior.
parse_genre_df = pd.DataFrame()

for row_number, movie in movie_genres.iterrows():
    genre_list = []
#     print(type(movie_genres))
#     print(type(movie_genres.to_string()))
    for genre_counter, genre in enumerate(literal_eval(movie['genres'])):
#   print(movie['id'], genre.get('name'))
        genre_list.append(genre.get('name'))
            
        

    # Associate the parsed genres with the movie id
    parse_genre_df = parse_genre_df.append({"id": movie['id'],
                                            'genre': ', '.join(map(str, genre_list))},
                                           ignore_index=True,
                                           sort=False)
   

In [17]:
# parse_genre_df.head()

In [18]:
prod_company = movies_df[["id", "production_companies"]]

In [19]:
x = movies_df["production_companies"][1]
x

"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"

In [20]:
eval(x)

[{'name': 'TriStar Pictures', 'id': 559},
 {'name': 'Teitler Film', 'id': 2550},
 {'name': 'Interscope Communications', 'id': 10201}]

In [21]:
eval(x)[0]["name"]

'TriStar Pictures'

In [22]:
[y["name"] for y in eval(x)]

['TriStar Pictures', 'Teitler Film', 'Interscope Communications']

In [23]:
all(map(lambda z: isinstance(z, dict), eval(x)))

True

In [24]:
isinstance(eval(x), list)

True

In [25]:
def fun2(x):
    try: 
        #r = eval(x)
        r = fun(x)
    except: 
        r = None
    return r
test = movies_df["production_companies"].apply(fun2)

In [26]:
test

0        None
1        None
2        None
3        None
4        None
         ... 
45461    None
45462    None
45463    None
45464    None
45465    None
Name: production_companies, Length: 45466, dtype: object

In [27]:
#better names? get_company_name 
#GOAL: select the name of the company from the original string
def fun(x): 
    if isinstance(x, str):
        x_eval = eval(x)
        if isinstance(x_eval, list):
            if all(map(lambda z: isinstance(z, dict), x_eval)):
                return [y["name"] for y in x_eval]
    else: 
        return None #"NA"

#GOAL: Applying 1st function while avoiding error messages / try_get_company_name
def fun2(x):
    try: 
        r = fun(x)
    except: 
        r = None #"NA"
    return r
    
movies_df["production_companies_clean"] = movies_df["production_companies"].apply(fun2)

In [28]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_clean
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,[Pixar Animation Studios]
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[TriStar Pictures, Teitler Film, Interscope Co..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Warner Bros., Lancaster Gate]"
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,[Twentieth Century Fox Film Corporation]
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[Sandollar Productions, Touchstone Pictures]"


In [29]:
test.values

array([None, None, None, ..., None, None, None], dtype=object)

In [30]:
list1 = [x for x in test.values if x is not None]

In [31]:
list2 = [y for x in list1 for y in x]
# if x is not None

In [32]:
set(list2)

set()

In [33]:
final = list(set(list2))

In [35]:
#[item for sublist in test.values for item in sublist]

In [36]:
# ##parse_companies_df = pd.DataFrame()

# ##for row_number, movie in prod_company.iterrows():
#     ##company_list = []
#     ##try:
#         for company_counter, company in enumerate(literal_eval(movie['production_companies'])):
#             company_list.append(company.get('name'))
#     except ValueError:
#         pass
        
#     parse_companies_df = parse_companies_df.append({'id': movie['id'],
#                                                    'production_companies': ', '.join(map(str, company_list))},
#                                            ignore_index=True,
#                                            sort=False)


In [37]:
# parse_companies_df.head()

In [38]:
movies_df.drop(['genres', 'production_companies'], axis=1, inplace = True)

In [39]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'production_companies_clean'],
      dtype='object')

In [41]:
#movies_df = movies_df.merge(parse_companies_df, on="id")

In [ ]:
#movies_df = movies_df.merge(parse_genre_df, on="id")

In [42]:
movies_df.to_csv('new_movie_metadata.csv', index=False)

In [43]:
movies_df = movies_df.drop(['adult', 'belongs_to_collection', 'homepage', 'original_language',
                'original_title', 'overview', 'popularity', 'poster_path', 'production_countries',
                'runtime', 'spoken_languages', 'status', 'tagline', 'video', 'vote_count'], axis=1)

In [44]:
movies_df.head()

,budget,id,imdb_id,release_date,revenue,title,vote_average,production_companies_clean
0,30000000,862,tt0114709,1995-10-30,373554033.0,Toy Story,7.7,[Pixar Animation Studios]
1,65000000,8844,tt0113497,1995-12-15,262797249.0,Jumanji,6.9,"[TriStar Pictures, Teitler Film, Interscope Co..."
2,0,15602,tt0113228,1995-12-22,0.0,Grumpier Old Men,6.5,"[Warner Bros., Lancaster Gate]"
3,16000000,31357,tt0114885,1995-12-22,81452156.0,Waiting to Exhale,6.1,[Twentieth Century Fox Film Corporation]
4,0,11862,tt0113041,1995-02-10,76578911.0,Father of the Bride Part II,5.7,"[Sandollar Productions, Touchstone Pictures]"


In [45]:
#renaming the columns
movies_df = movies_df.rename(columns={'budget': 'production_budget', 'id': 'movie_id', 'title': 'movie_name', 
                                      'vote_average': 'imdb_rating'})

In [46]:
# Drop all blank movie ID's

movies_df = movies_df.dropna(subset=['movie_id'])

In [47]:
# Removing the 0's before movie_id 

movies_df['movie_id'] = movies_df['movie_id'].astype(str).str.replace('\D+', '')

In [48]:
# Setting movie_id to int

movies_df['movie_id'] = movies_df['movie_id'].apply(lambda x: int(x))

In [49]:
#working on the RATING CSV here
ratings_df = ratings_df.groupby(['movieId'], as_index = False).mean()

In [50]:
# Add rating to movies_df

movies_df['rating'] = movies_df.movie_id.map(ratings_df.set_index('movieId')['rating'].to_dict())

In [51]:
movies_df.head()

,production_budget,movie_id,imdb_id,release_date,revenue,movie_name,imdb_rating,production_companies_clean,rating
0,30000000,862,tt0114709,1995-10-30,373554033.0,Toy Story,7.7,[Pixar Animation Studios],3.454545
1,65000000,8844,tt0113497,1995-12-15,262797249.0,Jumanji,6.9,"[TriStar Pictures, Teitler Film, Interscope Co...",4.125000
2,0,15602,tt0113228,1995-12-22,0.0,Grumpier Old Men,6.5,"[Warner Bros., Lancaster Gate]",NaN
3,16000000,31357,tt0114885,1995-12-22,81452156.0,Waiting to Exhale,6.1,[Twentieth Century Fox Film Corporation],NaN
4,0,11862,tt0113041,1995-02-10,76578911.0,Father of the Bride Part II,5.7,"[Sandollar Productions, Touchstone Pictures]",NaN


In [52]:
#Yearly Production Studio code here

In [53]:
release_date = movies_df[['movie_id','release_date']].head()

In [54]:
#a for loop to pick up dates in the excel that are messy.
year_list = []

for row_number, date in movies_df["release_date"].items():
    if isinstance(date, float) or len(date) <=3:
        year_list.append(None)
    else:
        try:
            date_year = datetime.strptime(date, "%m/%d/%Y")
            year_list.append(date_year.year)
        except ValueError:
            date_year = datetime.strptime(date, "%Y-%m-%d")
            year_list.append(date_year.year)

In [55]:
#new column, who dis?
movies_df["release_year"] = year_list

In [56]:
movies_df[["release_date","release_year"]]

,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0
...,...,...
45461,NaN,NaN
45462,2011-11-17,2011.0
45463,2003-08-01,2003.0
45464,1917-10-21,1917.0


In [58]:
production = movies_df[["production_companies_clean","revenue","production_budget","movie_name","release_year"]].copy()

In [60]:
production = production.dropna(subset=['movie_name','production_companies_clean'])

In [61]:
top = production.sort_values(by = ["revenue"],ascending = False)

In [63]:
#filtering production companies with Marvel in string
#Marvel = production[production['production_companies_clean'].str.contains('Marvel')]

In [ ]:
#looking at the most recent film in the table
#sort_year = Marvel.sort_values(by = ["release_year"],ascending = False)

In [ ]:
# #sorting by title since this table didn't have an index prior. dropped films that had 0 info
# #top_ten = sort_year.set_index('movie_name').drop(['Team Thor','Marvel Studios: Assembling a Universe',
#                                              'Marvel One-Shot: All Hail the King',
#                                              'Iron Man & Captain America: Heroes United',
#                                              'Marvel: 75 Years, From Pulp to Pop!','Iron Man & Hulk: Heroes United',
#                                              'Marvel One-Shot: Agent Carter','Marvel One-Shot: Item 47',
#                                              'Thor: Tales of Asgard','Planet Hulk'])

In [ ]:
#reseting the index
#top_ten.reset_index(level='movie_name')

In [ ]:
#convert budget to float since sum below wouldnt work as a object
#top_ten["production_budget"] = top_ten["production_budget"].astype(float)

In [ ]:
# revenue_sum = top_ten.groupby(['release_year'])['revenue'].sum()

# budget_sum = top_ten.groupby(['release_year'])['production_budget'].sum()

In [ ]:
#merge_inner = pd.merge(left=revenue_sum, right=budget_sum, left_on='release_year', right_on='release_year')

In [ ]:
# #final table with total revenue and budget per year
# merge_inner.sort_values(by = ["release_year"],ascending = False)

In [65]:
movies_df.head()

,production_budget,movie_id,imdb_id,release_date,revenue,movie_name,imdb_rating,production_companies_clean,rating,release_year
0,30000000,862,tt0114709,1995-10-30,373554033.0,Toy Story,7.7,[Pixar Animation Studios],3.454545,1995.0
1,65000000,8844,tt0113497,1995-12-15,262797249.0,Jumanji,6.9,"[TriStar Pictures, Teitler Film, Interscope Co...",4.125000,1995.0
2,0,15602,tt0113228,1995-12-22,0.0,Grumpier Old Men,6.5,"[Warner Bros., Lancaster Gate]",NaN,1995.0
3,16000000,31357,tt0114885,1995-12-22,81452156.0,Waiting to Exhale,6.1,[Twentieth Century Fox Film Corporation],NaN,1995.0
4,0,11862,tt0113041,1995-02-10,76578911.0,Father of the Bride Part II,5.7,"[Sandollar Productions, Touchstone Pictures]",NaN,1995.0


In [68]:
table_movies_df = movies_df[["movie_id", "movie_name", "release_year", 
                            "rating", "revenue", "production_companies_clean", "production_budget"]]
table_movies_df

,movie_id,movie_name,release_year,rating,revenue,production_companies_clean,production_budget
0,862,Toy Story,1995.0,3.454545,373554033.0,[Pixar Animation Studios],30000000
1,8844,Jumanji,1995.0,4.125000,262797249.0,"[TriStar Pictures, Teitler Film, Interscope Co...",65000000
2,15602,Grumpier Old Men,1995.0,NaN,0.0,"[Warner Bros., Lancaster Gate]",0
3,31357,Waiting to Exhale,1995.0,NaN,81452156.0,[Twentieth Century Fox Film Corporation],16000000
4,11862,Father of the Bride Part II,1995.0,NaN,76578911.0,"[Sandollar Productions, Touchstone Pictures]",0
...,...,...,...,...,...,...,...
45461,439050,Subdue,NaN,NaN,0.0,[],0
45462,111109,Century of Birthing,2011.0,3.000000,0.0,[Sine Olivia],0
45463,67758,Betrayal,2003.0,NaN,0.0,[American World Pictures],0
45464,227506,Satan Triumphant,1917.0,NaN,0.0,[Yermoliev],0


In [ ]:
table_production_companies_df = 